## IDC409: Database Creation

In [1]:
from mysql.connector import connect
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import time

##### Info: 
- Reference: https://realpython.com/python-mysql/#installing-mysql-server-and-mysql-connectorpython
- To start MYSQL service: ```service mysql start```
- Source website: https://www.mdpi.com/

### SQL Queries

In [2]:
create_table_query = """
CREATE TABLE articles(
    id INT AUTO_INCREMENT PRIMARY KEY,
    title VARCHAR(1000),
    authors VARCHAR(1000),
    journal VARCHAR(1000),
    published_date DATE,
    doi VARCHAR(500),
    abstract VARCHAR(5000)
)"""

insert_query = """
INSERT INTO articles
(title, authors, journal, published_date, doi, abstract)
VALUES (%s, %s, %s, %s, %s, %s)
"""

### Helper functions to clean data

In [3]:
#Helper functions to clean and extract relevant data

url = re.compile(" (https://doi.org/.*?) ")
date = re.compile(" (\d{2} \w{3} \d{4})")

def cleanAuthor(auth):
    auth = re.sub("\\nby\\n", "", auth)
    auth = re.sub(" and ", ", ", auth)
    return auth

def cleanDOI(doi):
    return url.search(doi).group(1)

def cleanDate(doi):
    date_obj = datetime.strptime(date.search(doi).group(1), '%d %b %Y')
    return date_obj.strftime('%Y-%m-%d')

def cleanJournal(doi):
    journ = re.compile("(.*);")
    return journ.search(doi).group(1)

def cleanAbstract(abst):
    abst = re.sub("(Full article|\\n)", "", abst)
    return abst

### Get maximum number of pages

In [10]:
def getMaxPages(max_pg, kw = "", auth = ""):
    
    #replace space with plus in the queries
    kw = re.sub(" ", "+", kw)
    auth = re.sub("[ ,]", "+", auth)
    
    #Webscrape the results page for list of articles
    url = "https://www.mdpi.com/search?sort=pubdate&page_no=1"\
           +"&page_count=15&year_from=1996&year_to=2021&q="\
           + kw + "&authors=" + auth + "&view=default"
     
    print(url)
    p=requests.get(url) #get the html content of the page
    s=BeautifulSoup(p.content, "html.parser") #parse the page
    
    #get the max page number
    pg = re.compile("of (\d+).")
    pgtext = s.find("div", {"class":"columns large-6 medium-6 small-12"}).text

    return min(max_pg, int(pg.search(pgtext).group(1)))

### Main code to scrape the data and fill into SQL

In [33]:
def fillDatabase(pg, kw = "", auth = ""):
    
    #replace space with plus in the queries
    kw = re.sub(" ", "+", kw)
    auth = re.sub("[ ,]", "+", auth)
    
    #Webscrape the results page for list of articles
    url = "https://www.mdpi.com/search?sort=pubdate&page_no="\
           + str(pg) +"&page_count=15&year_from=1996&year_to=2021&q="\
           + kw + "&authors=" + auth + "&view=default"
            
    p=requests.get(url) #get the html content of the page
    s=BeautifulSoup(p.content, "html.parser") #parse the page

    #Extract the data
    titles = [title.text for title in s.select("a.title-link")]
    raw_authors = [author.text for author in s.find_all("div", {"class": "authors"})]
    raw_dois = [doi.text for doi in s.find_all("div", {"class": "color-grey-dark"})]
    raw_abstracts = [abstract.text for abstract in s.find_all("div", {"class": "abstract-full"})]
    
    #Clean the data
    authors = [cleanAuthor(author) for author in raw_authors]
    dois = [cleanDOI(doi) for doi in raw_dois]
    journals = [cleanJournal(doi) for doi in raw_dois]
    dates = [cleanDate(doi) for doi in raw_dois]
    abstracts = [cleanAbstract(abstract) for abstract in raw_abstracts]
    
    #Create tuples of records (title, authors, journal, date, doi, abstract)
    records = []
    for i in range(len(titles)):
        records.append((titles[i], authors[i], journals[i], dates[i], dois[i], abstracts[i]))
            
    #Insert data in SQL table
    with connection.cursor() as cursor:
        cursor.executemany(insert_query, records)
        connection.commit()
        
    #Delay to avoid being blocked by the web server
    time.sleep(2.5)
    
    #Progress indicator
    print("Page " + str(pg) + " completed...")

### Driver code, putting everything together

In [34]:
def driver(pg, kw = "", auth = ""):
    
    pg = getMaxPages(pg, kw, auth)
    
    for page in range(1, pg + 1):
        fillDatabase(page)

### Connect with the MySQL database

In [7]:
connection = connect(
        host="localhost",
        user="idc409",
        password="pwd", database="test")

In [37]:
## Drop table; in case wrong entries were filled
# with connection.cursor() as cursor:
#     cursor.execute("DROP TABLE articles")
#     connection.commit()
    
# Create table
with connection.cursor() as cursor:
    cursor.execute(create_table_query)
    connection.commit()

### Run the script

In [38]:
driver(10, kw = "", auth = "")

https://www.mdpi.com/search?sort=pubdate&page_no=1&page_count=15&year_from=1996&year_to=2021&q=&authors=&view=default
Page 1 completed...
Page 2 completed...
Page 3 completed...
Page 4 completed...
Page 5 completed...
Page 6 completed...
Page 7 completed...
Page 8 completed...
Page 9 completed...
Page 10 completed...
